In [1]:
# BEFORE RUNNING THE CELLS, PLEASE GO TO "EDIT" -> "NOTEBOOK SETTINGS" -> AND SET "HARDWARE ACCELERATOR" TO GPU
# import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
 
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import keras_metrics as km

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import multilabel_confusion_matrix

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dropout, Flatten, Dense, Activation, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Dropout
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler

import numpy as np 
import matplotlib.pyplot as plt 
import argparse
import os
import cv2 
import random
import sys
from PIL import Image
import pickle
import math

In [2]:
DATADIR = "./finaldata" 
CATEGORIES = ["Boring", "Interesting"]
training_data = []
seq_len = 41
IMG_SIZE = 64

In [3]:
def frame_extraction(file_path, file_name):
    frames_list = []
    for x in range(80, 121):
        img_array = cv2.imread(file_path + "/" + file_name + str(x) + ".png", cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
        frames_list.append(image)
    return frames_list

In [4]:
def create_data(input_dir):
    X = []
    Y = []
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        for image in sorted(os.listdir(path)): 
            if ".png" not in image or "80" not in image:
                continue
            file_name = image[0:-6]
            frames = frame_extraction(path, file_name)
            X.append(frames)
            y = [0] * 2
            y[CATEGORIES.index(category)] = 1
            Y.append(y)
    X = np.asarray(X)
    Y = np.asarray(Y)
    return X, Y

In [5]:
X, Y = create_data(DATADIR)
X = X.reshape(list(X.shape) + [1])
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, shuffle=True, random_state=0)

In [6]:
X.shape

(2100, 41, 300, 300)

In [11]:
model = Sequential()
model.add(ConvLSTM2D(filters = 64, kernel_size = (3, 3), return_sequences = False, data_format = "channels_last", input_shape = X.shape[1:]))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(2, activation = "softmax"))

In [12]:
model.summary()
opt = keras.optimizers.SGD(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])
earlystop = EarlyStopping(patience=7)
callbacks = [earlystop]
history = model.fit(x = X_train, y = y_train, epochs=20, batch_size = 8 , shuffle=True, validation_split=0.2, callbacks=callbacks)
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis = 1)
y_test = np.argmax(y_test, axis = 1)
print(classification_report(y_test, y_pred))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_1 (ConvLSTM2D)  (None, 298, 298, 64)      150016    
_________________________________________________________________
dropout_2 (Dropout)          (None, 298, 298, 64)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 5683456)           0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               1454964992
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 1542      
Total params: 1,455,116,550
Trainable params: 1,455,116,550
Non-trainable params: 0
____________________________________